In [2]:
# Step 1: Importing Libraries
import json
import numpy as np
import matplotlib.pyplot as plt  # For visualization
import seaborn as sns  # For better visualizations

In [3]:
training_challenges_path = "/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json"
training_solutions_path = "/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json"
test_challenges_path = "/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json"

In [4]:
# Step 2: Load Data
def load_data(training_challenges_path, training_solutions_path, test_challenges_path):
    with open(training_challenges_path) as f:
        training_challenges = json.load(f)
    with open(training_solutions_path) as f:
        training_solutions = json.load(f)
    with open(test_challenges_path) as f:
        test_challenges = json.load(f)
    return training_challenges, training_solutions, test_challenges

In [5]:
# Apply transformations: repeat, mirror, color change, symmetry
def apply_transformations(input_grid):
    input_array = np.array(input_grid)
    
    # Step 1: Repeat the grid (simple expansion)
    repeated_grid = np.tile(input_array, (2, 2))
    
    # Step 2: Mirror - Reflect the grid horizontally and vertically
    mirrored_horizontal = np.fliplr(input_array)
    mirrored_vertical = np.flipud(input_array)
    
    # Step 3: Color change - Example: Replace '2' with '8'
    color_changed_grid = np.where(input_array == 2, 8, input_array)
    
    # Step 4: Symmetry - Combine original and mirrored grids
    symmetrical_grid = np.hstack((input_array, mirrored_horizontal))
    symmetrical_grid = np.vstack((symmetrical_grid, np.hstack((mirrored_vertical, input_array))))
    
    # Ensure all grids are of the same size by trimming or padding
    target_size = input_array.shape  # Original grid size
    
    def resize_grid(grid, target_size):
        grid = np.array(grid)
        current_size = grid.shape
        if current_size != target_size:
            return np.resize(grid, target_size)
        return grid
    
    return {
        "repeated": resize_grid(repeated_grid, target_size).tolist(),
        "mirrored_horizontal": resize_grid(mirrored_horizontal, target_size).tolist(),
        "mirrored_vertical": resize_grid(mirrored_vertical, target_size).tolist(),
        "color_changed": resize_grid(color_changed_grid, target_size).tolist(),
        "symmetrical": resize_grid(symmetrical_grid, target_size).tolist()
    }

In [7]:
# Generate submission for test set
def generate_submission(test_challenges, output_path="submission.json"):
    submission = {}
    
    for task_id, task_data in test_challenges.items():
        input_grid = task_data["test"][0]["input"]  # Single test case per task
        transformed = apply_transformations(input_grid)
        
        # Ensure attempts are the same size as the original input grid
        submission[task_id] = [{
            "attempt_1": transformed["symmetrical"],  # or any transformation of your choice
            "attempt_2": transformed["color_changed"]  # or another transformation
        }]
    
    # Ensure the submission is valid before saving
    for task_id, outputs in submission.items():
        if len(outputs) != 1 or not outputs[0].get("attempt_1") or not outputs[0].get("attempt_2"):
            print(f"Warning: Task {task_id} has an invalid format. Check your transformations.")
    
    # Save submission to JSON
    with open(output_path, 'w') as f:
        json.dump(submission, f, indent=4)  # Indent for readability
    print(f"Submission saved to {output_path}")

In [8]:
# Load data
training_challenges, training_solutions, test_challenges = load_data(training_challenges_path, training_solutions_path, test_challenges_path)

In [9]:
generate_submission(test_challenges)

Submission saved to submission.json
